In [26]:
import pandas as pd

In [27]:
data_5s = pd.read_json("data_5scheduler.json")
data_5s = data_5s.dropna(subset=["description", "title"]) # looks like nothing actually gets dropped by this
data_5s.head(10)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd,300,[Staff],,,,False,0
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd,300,[Anup Gampa],,,,True,0
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...,HarveyMudd,300,[Balseiro],,,,False,0
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...,HarveyMudd,300,[de Laet],,,,False,0
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...,HarveyMudd,300,[Marianne De Laet],,,,True,0
5,War and Conflict,ANTH-115-HM,“The wings of the butterfly—that cause the hur...,HarveyMudd,300,[de Laet],,,,False,0
6,Rationalities,ANTH-134-HM,What does it mean to be rational? Does it mean...,HarveyMudd,300,[de Laet],Offered alternate years,Any introductory course in anthropology or any...,,False,0
7,A History of Landscape Photography,ARHI-131-HM,This course explores how photographic landscap...,HarveyMudd,300,[Fandell],,,,False,0
8,Modern and Contemporary Art Practices,ART-002-HM,This class is an experimental lecture style ar...,HarveyMudd,300,[Fandell],,,,False,0
9,Photography,ART-033-HM,Approaching the medium from an artistic perspe...,HarveyMudd,300,[Fandell],,ART002 HM,,False,150


In [28]:
data_5s.describe()

,credits,fee
count,4443.000000,4443.000000
mean,85.253657,0.082152
std,70.623956,2.864374
min,0.000000,0.000000
25%,25.000000,0.000000
50%,100.000000,0.000000
75%,100.000000,0.000000
max,600.000000,150.000000


In [29]:
data_stripped = data_5s.drop(["source", "credits", "instructors", "offered", "prerequisites", "corequisites", "currently_offered", "fee"], axis=1)

In [30]:
data_stripped.head()

,title,identifier,description
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...


In [31]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize
nltk.download('punkt')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wwsam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wwsam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [32]:
data_tokenized = data_stripped.copy()
data_tokenized["title_tokenized"] = data_tokenized.title.apply(lambda x: [i.lower() for i in word_tokenize(x) if i not in stop_words and i not in string.punctuation])
data_tokenized["description_tokenized"] = data_tokenized.description.apply(lambda x: [i.lower() for i in word_tokenize(x) if i not in stop_words and i not in string.punctuation])
data_tokenized["tokens"] = data_tokenized.title_tokenized + data_tokenized.description_tokenized
data_tokenized.drop(["title", "description", "title_tokenized", "description_tokenized"], axis=1, inplace=True)
data_tokenized.head()

,identifier,tokens
0,AMST-103-HM,"[introduction, american, cultures, an, interdi..."
1,AMST-115-HM,"[print, american, culture, covers, numerous, d..."
2,AMST-120-HM,"[hyphenated, americans, a, focus, experience, ..."
3,ANTH-110-HM,"[life, knowledge, belief, cultural, practices,..."
4,ANTH-111-HM,"[introduction, anthropology, science, technolo..."


In [33]:
from wikipedia2vec import Wikipedia2Vec
wv = Wikipedia2Vec.load("./enwiki_20180420_100d.pkl")
wv.get_word_vector("cultures")

C:\Users\wwsam\AppData\Local\Programs\Python\Python38\lib\site-packages\joblib\numpy_pickle.py:230: UserWarning: The memmapped array [[-0.12560804 -0.5235532  -0.13231167 ... -0.48633438  0.45111445
   0.11309847]
 [-0.12473089 -0.6455471   0.13622157 ... -0.6347396   0.5410465
   0.32159257]
 [-0.05129661 -0.4987064  -0.09563554 ... -0.30148467  0.24280211
  -0.08308805]
 ...
 [-1.0583687  -0.83086014 -0.4427682  ... -1.0160557  -0.02469787
   0.99688977]
 [-1.0772408  -0.32139003 -0.79673034 ... -1.3150353  -0.12155519
   0.57841384]
 [-0.86319923 -0.5425662  -0.5506818  ... -1.1069874   0.26995388
   0.43403685]] loaded from the file ./enwiki_20180420_100d.pkl is not not bytes aligned. This may cause segmentation faults if this memmapped array is used in some libraries like BLAS or PyTorch. To get rid of this warning, regenerate your pickle file with joblib >= 1.2.0. See https://github.com/joblib/joblib/issues/563 for more details
  warnings.warn(message)
C:\Users\wwsam\AppData\Loca

memmap([-0.25236896,  0.42014414, -0.08391619, -0.3252141 ,  0.42072064,
        -0.10993782,  0.0665969 , -0.20184374,  0.11382752,  0.2525333 ,
        -0.31821615,  0.03052388, -0.01315741, -0.2601099 ,  0.37494594,
         0.40026274,  0.15928762, -0.14020453,  0.0103266 ,  0.14715648,
        -0.21889728,  0.05747881, -0.1686419 ,  0.30100852, -0.15160422,
        -0.35125345,  0.50124305, -0.68395364,  0.20568438,  0.26270363,
         0.35114944,  0.5416242 ,  0.47201478,  0.8405194 ,  0.79017633,
         0.22338173, -0.00924228,  0.20680006,  0.42482582,  0.03633159,
         0.0314093 ,  0.48311287, -0.08734226, -0.17139564, -0.62047905,
        -0.57598805, -0.49039215, -0.15083963, -0.14144881,  0.47007284,
         0.01941797, -0.08926544, -0.23882082,  0.17695946,  0.00842945,
         0.08630176,  0.06666368, -0.5945753 , -0.7459417 ,  0.7895723 ,
         0.1397453 , -0.43910602, -0.36991954,  0.22501871,  0.8984134 ,
        -0.7012016 , -0.37027422, -0.33159506, -0.5

In [34]:
print(bool(wv.get_word("habada")))

False


In [46]:
def get_average_vector(tokens):
    vectors = [wv.get_word_vector(i) for i in tokens if wv.get_word(i)]
    average_vector = sum(vectors)/len(vectors)
    return average_vector

data_vectorized = data_tokenized.copy()
data_vectorized["vector"] = data_vectorized.tokens.apply(get_average_vector)
data_vectorized.drop("tokens", axis=1, inplace=True)
data_vectorized.head()

,identifier,vector
0,AMST-103-HM,"[-0.17634316, 0.3884739, -0.053265188, -0.0645..."
1,AMST-115-HM,"[0.12168898, 0.40610662, 0.115964696, 0.191182..."
2,AMST-120-HM,"[-0.15255982, 0.2974986, -0.019600628, -0.0695..."
3,ANTH-110-HM,"[-0.05883435, 0.24085796, -0.13384405, -0.0150..."
4,ANTH-111-HM,"[-0.13835831, 0.25955617, -0.13408767, 0.09626..."


In [78]:
list(data_vectorized[data_vectorized['identifier'] == 'ENGR-083-HM'].vector)[0]

array([-0.06843871,  0.2934974 , -0.10484475,  0.01260288, -0.2756224 ,
       -0.01086704,  0.13048644,  0.1994828 ,  0.3598929 , -0.1401149 ,
        0.09570518, -0.11910256,  0.01974999, -0.4099996 ,  0.2375624 ,
       -0.3198687 , -0.0411201 , -0.26285422,  0.06976139, -0.12402815,
        0.32310665,  0.08096309, -0.06111833, -0.04905834,  0.03830248,
       -0.17483515, -0.09655087, -0.36405155,  0.32721546,  0.2627999 ,
        0.37410125, -0.05960902,  0.06767824,  0.3198454 ,  0.20534915,
       -0.08779518, -0.11315194, -0.18928753,  0.14137784,  0.0953515 ,
        0.02771445,  0.09000498, -0.14935112,  0.3373088 , -0.24156004,
       -0.24550517, -0.27677396,  0.07883749, -0.23115271,  0.11450494,
       -0.0828912 ,  0.33793497, -0.21120465,  0.31029367,  0.2857777 ,
       -0.11135299, -0.0298343 , -0.06467684, -0.08048104,  0.53411365,
       -0.3367021 , -0.07817362, -0.3460646 ,  0.19970177,  0.67206866,
       -0.4962066 , -0.326636  , -0.09530025, -0.27015835, -0.48

In [79]:
import numpy as np

def get_similar_courses(search_course, data_vectorized, data_5s):
    this_vector = list(data_vectorized[data_vectorized['identifier'] == search_course].vector)[0]
    data_search = data_vectorized[data_vectorized.identifier != search_course]
    data_info = data_5s.copy()
    data_info["cosine_sim"] = data_search.apply(lambda x: np.inner(x.vector, this_vector) / (np.linalg.norm(x.vector) * np.linalg.norm(this_vector)), axis=1)
    data_info.sort_values("cosine_sim", inplace=True, ascending=False)
    return data_info.head(10)

In [80]:
get_similar_courses("AMST-103-HM", data_vectorized, data_5s)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,cosine_sim
383,Introduction to American Cultures,AMST-103-JT,"This course, taught by an intercollegiate facu...",ClaremontMckenna,100,[],Every spring,,,False,0,0.958275
3589,Introduction to American Culture,AMST-103-JT,"This course, taught by an intercollegiate facu...",Pitzer,100,[Scott Sawyer],,,,True,0,0.957345
3710,Introduction to Asian American Studies,ASAM-101-PZ,Introduction to the field of Asian American St...,Pitzer,100,[Rosanna Simons],,,,True,0,0.938488
2659,Introduction to American Cultures,AMST-103-SC,This class analyzes the histories and cultures...,Scripps,100,[],Spring,,,False,0,0.924607
1451,Senior Seminar in Classics,CLAS-190-PO,A seminar for review and discussion of major t...,Pomona,100,[Staff],Each fall.,,,False,0,0.922195
1839,Afro-Latin American (CP),HIST-136-PO,This class examines the history of socio-cultu...,Pomona,100,[A. Mayes],Last offered spring 2019.,,,False,0,0.921884
820,Interdisciplinary Special Topics,ID-150-JT,The scholar and the athlete have long existed ...,ClaremontMckenna,100,"[Seth Lobis, Paul Settles]",Every other year,,,True,0,0.921152
4338,Seminar in African American Psychology,PSYC-188-AF,Critically examines contemporary literature in...,Pitzer,0,[],,,,False,0,0.918434
4405,Teaching as Social Change,SOC-188-AA,This seminar will explore theoretical work on ...,Pitzer,0,[],,,,False,0,0.917904
3030,Postwar American Poetry,ENGL-160S-SC,This course will examine a variety of influent...,Scripps,100,[],,,,False,0,0.917529


In [81]:
get_similar_courses("ENGR-083-HM", data_vectorized, data_5s)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,cosine_sim
197,Advanced Structural Dynamics,ENGR-278-HM,Free and forced response of continuous systems...,HarveyMudd,300,[Philip D. Cha],"Spring, alternate years",ENGR171 HM,,True,0,0.964813
182,Dynamics of Rigid Bodies,ENGR-175-HM,"Kinematics, mass distribution, and kinetics of...",HarveyMudd,300,[Bassman],,,ENGR083 HM,False,0,0.963676
183,Numerical Methods in Engineering,ENGR-176-HM,This course focuses on the application of a va...,HarveyMudd,300,"[Cha, Wang]","Spring, alternate years",ENGR072 HM,,False,0,0.960987
2337,Introduction to Physical Hydrodynamics,PHYS-165-PO,This course introduces the ideas underlying fl...,Pomona,100,[D. Whitaker],Last offered spring 2018.,,PHYS 125 PO or 5C equivalent,False,0,0.959234
1964,Differential Equations and Modeling,MATH-102-PO,Introduction to the theory of ordinary differe...,Pomona,100,[Maryann Hohn],Each semester.,MATH 032 PO or MATH 067 PO ; and MATH 060 PO o...,,True,0,0.958644
272,Graduate Partial Differential Equations,MATH-184-HM,Advanced topics in the study of linear and non...,HarveyMudd,300,"[Bernoff, Castro, Jacobsen]","Spring, alternate years",(MATH115 HM and MATH131 HM ) or MATH180 HM ; ...,,False,0,0.955719
327,Theoretical Mechanics,PHYS-111-HM,The application of mathematical methods to the...,HarveyMudd,300,[Leah Mendelson],Fall,"PHYS023 HM , PHYS024 HM , and MATH082 HM",,True,0,0.953914
271,Dynamical Systems,MATH-181-HM,Existence and uniqueness theorems for systems ...,HarveyMudd,300,[Andrew J. Bernoff],"Jointly; Fall semester at Pomona, Spring semes...",MATH115 HM or MATH180 HM,,True,0,0.950623
256,Stochastic Processes,MATH-156-HM,This course is particularly well-suited for th...,HarveyMudd,300,"[Benjamin, Martonosi, Staff (CMC)]","Jointly; Fall, alternate years at HMC",MATH073 HM and MATH157 HM,,False,0,0.948727
169,Fluid Mechanics,ENGR-131-HM,Integrated approach to the subjects of fluid m...,HarveyMudd,300,[Lape],Fall,ENGR083 HM,,False,0,0.947609
